In [ ]:
from ultralytics import YOLO

Predict with the model

In [ ]:
# Load a model
model = YOLO("D:\yolov8\guaTiao/best.pt")  # load an official model


# Predict with the model
results = model("D:\yolov8\guaTiao\阿尔卑斯----OK/0b24665f-b478-4ab5-a423-87d978d15e5f.jpg")  # predict on an image

Train model

In [ ]:
# Train the model

model = YOLO("yolov8n-obb.yaml")  
# load a pretrained model (recommended for training)
# model = YOLO("yolov8n.pt")  
# build from YAML and transfer weights
# model = YOLO("yolov8n.yaml").load("yolov8n.pt")  
model.train(data='dota8-obb.yaml', epochs=10, batch=32, workers=4)


Validate the model

In [ ]:
# Validate the model
def main():
    model = YOLO("D:\yolov8\guaTiao/best.pt")
    model.val(data='dota8-obb.yaml', imgsz=1024, batch=4, workers=4)
 
    # 如果你有test就用下面的语句
    # model.val(data='dota8-obb.yaml',split='test', imgsz=1024, batch=4, workers=4)
 
if __name__ == '__main__':
    main()

OBB.XML ----> YOLO.TXT

In [ ]:
# 功能描述   ：把rolabelimg标注的xml文件转换成dota能识别的xml文件，
#             再转换成dota格式的txt文件
#            把旋转框 cx,cy,w,h,angle，或者矩形框cx,cy,w,h,转换成四点坐标x1,y1,x2,y2,x3,y3,x4,y4
# !!!!!!!!!! : 需要修改的内容 
# 1.line 12 的标签，这里的顺序需要和converter.py中的convert_dota_to_yolo_obb中的class_mapping顺序保持一致
# 2.line 164，165，166中的文件夹名字，这里需要先把rolabelimg打完的xml标签放入/“数据集名字”/xml中，并在/“数据集名字”/下创建DOTA_xml和DOTA_txt，转完后讲DOTA_txt放到数据集的labels/train_original和labels/val_original下，具体拆分比例自己定，然后运行下一个jupyter cell


# 更新！！！直接修改标签+202行的数据集地址就行，转换和DOTA文件夹都会自动创建，但！仍然要放在xml文件夹下
import os
import shutil
import xml.etree.ElementTree as ET
import math
from pathlib import Path
import os
import cv2
from ultralytics.utils import LOGGER, TQDM

 
cls_list = ['0']  # 修改为自己的标签

data_path = r'D:\code\datasets\zhuoshangbingxiang\shangpin'



def convert_dota_to_yolo_obb(dota_root_path: str):
    
    # Class names to indices mapping
    class_mapping = {
        "0": 0,
    }
    
    dota_root_path = Path(dota_root_path)
  

    def convert_label(image_name, image_width, image_height, orig_label_dir, save_dir, image_path):
        """Converts a single image's DOTA annotation to YOLO OBB format and saves it to a specified directory."""
        orig_label_path = orig_label_dir / f"{image_name}.txt"
        save_path = save_dir / f"{image_name}.txt"

        try:
            with orig_label_path.open("r") as f, save_path.open("w") as g:
                lines = f.readlines()
                for line in lines:
                    parts = line.strip().split()
                    if len(parts) < 9:
                        continue
                    class_name = parts[8]
                    class_idx = class_mapping[class_name]
                    coords = [float(p) for p in parts[:8]]
                    normalized_coords = [
                        coords[i] / image_width if i % 2 == 0 else coords[i] / image_height for i in range(8)
                    ]
                    formatted_coords = ["{:.6g}".format(coord) for coord in normalized_coords]
                    g.write(f"{class_idx} {' '.join(formatted_coords)}\n")
        except FileNotFoundError:
            os.remove(image_path)
            print(f"DELETE!!!{image_path}")

    for phase in ["train", "val"]:
        image_dir = dota_root_path / "images" / phase
        orig_label_dir = dota_root_path / "labels" / f"{phase}_original"
        save_dir = dota_root_path / "labels" / phase

        save_dir.mkdir(parents=True, exist_ok=True)

        image_paths = list(image_dir.iterdir())
        for image_path in TQDM(image_paths, desc=f"Processing {phase} images"):
            # if image_path.suffix != ".jpg":
            #     continue
            image_name_without_ext = image_path.stem
            img = cv2.imread(str(image_path))
            h, w = img.shape[:2]
            convert_label(image_name_without_ext, w, h, orig_label_dir, save_dir, image_path)
 
def edit_xml(xml_file, dotaxml_file):
    """
    修改xml文件
    :param xml_file:xml文件的路径
    :return:
    """
 
    # dxml_file = open(xml_file,encoding='gbk')
    # tree = ET.parse(dxml_file).getroot()
 
    tree = ET.parse(xml_file)
    objs = tree.findall('object')
    for ix, obj in enumerate(objs):
        x0 = ET.Element("x0")  # 创建节点
        y0 = ET.Element("y0")
        x1 = ET.Element("x1")
        y1 = ET.Element("y1")
        x2 = ET.Element("x2")
        y2 = ET.Element("y2")
        x3 = ET.Element("x3")
        y3 = ET.Element("y3")
        # obj_type = obj.find('bndbox')
        # type = obj_type.text
        # print(xml_file)
 
        if (obj.find('robndbox') == None):
            obj_bnd = obj.find('bndbox')
            obj_xmin = obj_bnd.find('xmin')
            obj_ymin = obj_bnd.find('ymin')
            obj_xmax = obj_bnd.find('xmax')
            obj_ymax = obj_bnd.find('ymax')
            # 以防有负值坐标
            xmin = max(float(obj_xmin.text), 0)
            ymin = max(float(obj_ymin.text), 0)
            xmax = max(float(obj_xmax.text), 0)
            ymax = max(float(obj_ymax.text), 0)
            obj_bnd.remove(obj_xmin)  # 删除节点
            obj_bnd.remove(obj_ymin)
            obj_bnd.remove(obj_xmax)
            obj_bnd.remove(obj_ymax)
            x0.text = str(xmin)
            y0.text = str(ymax)
            x1.text = str(xmax)
            y1.text = str(ymax)
            x2.text = str(xmax)
            y2.text = str(ymin)
            x3.text = str(xmin)
            y3.text = str(ymin)
        else:
            obj_bnd = obj.find('robndbox')
            obj_bnd.tag = 'bndbox'  # 修改节点名
            obj_cx = obj_bnd.find('cx')
            obj_cy = obj_bnd.find('cy')
            obj_w = obj_bnd.find('w')
            obj_h = obj_bnd.find('h')
            obj_angle = obj_bnd.find('angle')
            cx = float(obj_cx.text)
            cy = float(obj_cy.text)
            w = float(obj_w.text)
            h = float(obj_h.text)
            angle = float(obj_angle.text)
            obj_bnd.remove(obj_cx)  # 删除节点
            obj_bnd.remove(obj_cy)
            obj_bnd.remove(obj_w)
            obj_bnd.remove(obj_h)
            obj_bnd.remove(obj_angle)
 
            x0.text, y0.text = rotatePoint(cx, cy, cx - w / 2, cy - h / 2, -angle)
            x1.text, y1.text = rotatePoint(cx, cy, cx + w / 2, cy - h / 2, -angle)
            x2.text, y2.text = rotatePoint(cx, cy, cx + w / 2, cy + h / 2, -angle)
            x3.text, y3.text = rotatePoint(cx, cy, cx - w / 2, cy + h / 2, -angle)
 
        # obj.remove(obj_type)  # 删除节点
        obj_bnd.append(x0)  # 新增节点
        obj_bnd.append(y0)
        obj_bnd.append(x1)
        obj_bnd.append(y1)
        obj_bnd.append(x2)
        obj_bnd.append(y2)
        obj_bnd.append(x3)
        obj_bnd.append(y3)
 
        tree.write(dotaxml_file, method='xml', encoding='utf-8')  # 更新xml文件

# 转换成四点坐标
def rotatePoint(xc, yc, xp, yp, theta):
    xoff = xp - xc;
    yoff = yp - yc;
    cosTheta = math.cos(theta)
    sinTheta = math.sin(theta)
    pResx = cosTheta * xoff + sinTheta * yoff
    pResy = - sinTheta * xoff + cosTheta * yoff
    return str(int(xc + pResx)), str(int(yc + pResy))
 
def totxt(xml_path, out_path):
    # 想要生成的txt文件保存的路径，这里可以自己修改
 
    files = os.listdir(xml_path)
    i = 0
    for file in files:
 
        tree = ET.parse(xml_path + os.sep + file)
        root = tree.getroot()
 
        name = file.split('.')[0]
 
        output = out_path + '\\' + name + '.txt'
        file = open(output, 'w')
        i = i + 1
        objs = tree.findall('object')
        for obj in objs:
            cls = obj.find('name').text
            box = obj.find('bndbox')
            x0 = int(float(box.find('x0').text))
            y0 = int(float(box.find('y0').text))
            x1 = int(float(box.find('x1').text))
            y1 = int(float(box.find('y1').text))
            x2 = int(float(box.find('x2').text))
            y2 = int(float(box.find('y2').text))
            x3 = int(float(box.find('x3').text))
            y3 = int(float(box.find('y3').text))
            if x0 < 0:
                x0 = 0
            if x1 < 0:
                x1 = 0
            if x2 < 0:
                x2 = 0
            if x3 < 0:
                x3 = 0
            if y0 < 0:
                y0 = 0
            if y1 < 0:
                y1 = 0
            if y2 < 0:
                y2 = 0
            if y3 < 0:
                y3 = 0
            for cls_index, cls_name in enumerate(cls_list):
                if cls == cls_name:
                    file.write("{} {} {} {} {} {} {} {} {} {}\n".format(x0, y0, x1, y1, x2, y2, x3, y3, cls, cls_index))
        file.close()
        # print(output)
        print(i)
 
def dir_exists(folder_path):
        # 判断文件夹是否存在
    if not os.path.exists(folder_path):
        # 如果不存在则创建文件夹
        os.makedirs(folder_path)
        print(f"文件夹 '{folder_path}' 不存在，已创建。")
    else:
        print(f"文件夹 '{folder_path}' 已存在。")
 
def copy_files_except_last_n(source_folder, destination_folder, n):
    try:
        # 获取源文件夹中的所有文件列表
        files = os.listdir(source_folder)
        
        # 确保文件列表按照文件名排序
        files.sort()
        
        # 选择要复制的文件
        files_to_copy = files[:-n]  # 复制除了最后 n 个文件之外的所有文件
        
        # 创建目标文件夹（如果不存在）
        os.makedirs(destination_folder, exist_ok=True)
        
        # 复制选定的文件到目标文件夹
        for file_name in files_to_copy:
            source_file = os.path.join(source_folder, file_name)
            destination_file = os.path.join(destination_folder, file_name)
            shutil.copy(source_file, destination_file)
        
        print(f"成功复制文件夹 '{source_folder}' 中除了最后 {n} 个文件之外的所有文件到 '{destination_folder}'")
    
    except FileNotFoundError:
        print(f"文件夹 '{source_folder}' 不存在或无法访问。")
    except OSError as e:
        print(f"复制文件时发生OSError: {e}")
 
if __name__ == '__main__':
    # -----**** 第一步：把xml文件统一转换成旋转框的xml文件 ****-----
    
    roxml_path = os.path.join(data_path, 'xml') # 存放roLabelImg标注的原文件的文件夹
    
    # 获取文件夹中所有文件
    files = os.listdir(data_path)

    # 提取jpg和xml文件名（不含扩展名）
    jpg_files = {os.path.splitext(file)[0] for file in files if file.endswith('.jpg')}
    xml_files = {os.path.splitext(file)[0] for file in files if file.endswith('.xml')}

    # 找到没有对应xml文件的jpg文件
    jpg_without_xml = jpg_files - xml_files

    # 删除没有对应xml文件的jpg文件
    for jpg in jpg_without_xml:
        jpg_path = os.path.join(data_path, f'{jpg}.jpg')
        os.remove(jpg_path)
        print(f'Deleted {jpg_path}')
    
    dotaxml_path = os.path.join(data_path, 'DOTA_xml') # 存放转换后DOTA数据集xml格式文件的文件夹
    out_path = os.path.join(data_path, 'DOTA_txt') # 存放转换后DOTA数据集txt格式文件的文件夹
    
    dir_exists(dotaxml_path)
    dir_exists(out_path)
    dir_exists(roxml_path)
    
    # 将所有xml文件移动到xml文件夹中
    for xml in xml_files:
        xml_path = os.path.join(data_path, f'{xml}.xml')
        shutil.move(xml_path, roxml_path)
        print(f'Moved {xml_path} to {roxml_path}')
    
    filelist = os.listdir(roxml_path)
 
    for file in filelist:
        edit_xml(os.path.join(roxml_path, file), os.path.join(dotaxml_path, file))
 
    # -----**** 第二步：把旋转框xml文件转换成txt格式 ****-----
    totxt(dotaxml_path, out_path)
    
    labels_train_path = os.path.join(data_path, 'labels/train_original')
    labels_val_path = os.path.join(data_path, 'labels/val_original')
    
    dir_exists(labels_train_path)
    dir_exists(labels_val_path)

    # 获取out_path文件夹中的所有txt文件
    txt_files = [file for file in os.listdir(out_path) if file.endswith('.txt')]

    # 按文件名排序以确保一致性
    txt_files.sort()

    # 将最后7个文件放到val文件夹中，其余的文件放到train文件夹中
    val_files = txt_files[-7:]
    train_files = txt_files[:-7]

    # 移动文件到对应的文件夹中
    for file in val_files:
        src_path = os.path.join(out_path, file)
        dst_path = os.path.join(labels_val_path, file)
        shutil.move(src_path, dst_path)
    print(f'Moved .txt from {out_path} to {labels_val_path}')

    for file in train_files:
        src_path = os.path.join(out_path, file)
        dst_path = os.path.join(labels_train_path, file)
        shutil.move(src_path, dst_path)
    print(f'Moved .txt from {src_path} to {dst_path}')
        
        
        
    
    images_train_path = os.path.join(data_path, 'images/train')
    images_val_path = os.path.join(data_path, 'images/val')
    
    dir_exists(images_train_path)
    dir_exists(images_val_path)    
    
    # 获取out_path文件夹中的所有txt文件
    jpg_files = [file for file in os.listdir(data_path) if file.endswith('.jpg')]

    # 按文件名排序以确保一致性
    jpg_files.sort()

    # 将最后7个文件放到val文件夹中，其余的文件放到train文件夹中
    val_files = jpg_files[-7:]
    train_files = jpg_files[:-7]

    # 移动文件到对应的文件夹中
    for file in val_files:
        src_path = os.path.join(data_path, file)
        dst_path = os.path.join(images_val_path, file)
        shutil.move(src_path, dst_path)
    print(f'Moved .jpg from {data_path} to {images_val_path}')

    for file in train_files:
        src_path = os.path.join(data_path, file)
        dst_path = os.path.join(images_train_path, file)
        shutil.move(src_path, dst_path)
    print(f'Moved .jpg from {data_path} to {images_train_path}')
        
    convert_dota_to_yolo_obb(data_path) # DOTA 格式数据集路径，里头包含 images和labels，labels中包含train_original和val_original
    
    shutil.rmtree(dotaxml_path)
    shutil.rmtree(out_path)
    shutil.rmtree(roxml_path)

Labelimg.json ——> YOLO.TXT

将所有 LabelMe JSON 文件和图片名字统一 放在labelme_json_dir下。脚本将按文件夹读取训练和验证数据集。运行这个 python 命令。

labelme2yolo --val_size 0.05 --json_dir D:\code\datasets\



In [ ]:
终端运行！！！labelme2yolo --val_size 0.05 --json_dir D:\code\datasets\zhuoshangbingxiang\huojia 

In [ ]:
import os
import shutil
import subprocess

# Define paths
data_path = 'data_path'
images_path = os.path.join(data_path, 'images')
labels_path = os.path.join(data_path, 'labels')
yolo_dataset_path = 'YOLODataset'

# Run labelme2yolo command
subprocess.run(['labelme2yolo', '--val_size', '0.05', '--json_dir', data_path])

# Delete original images and labels directories
shutil.rmtree(images_path)
shutil.rmtree(labels_path)

# Move YOLODataset content to data_path
for item in os.listdir(yolo_dataset_path):
    shutil.move(os.path.join(yolo_dataset_path, item), data_path)

# Delete YOLODataset directory
shutil.rmtree(yolo_dataset_path)

print("Process completed successfully.")


统计数据集中每个类别的数量

In [ ]:
# label count
import os
def labels_count(root):
    '''
    统计数据集中每个类别的数量
    args:
        root:yolog格式数据集的label路径
    return:
        labels_count:数据集中每个类别的数量
    '''
    labels_count = {}
    for dir in os.listdir(root):
        labels_dir_count = {}
        if dir in ['train', 'val']:
            label_dir = os.path.join(root, dir)
            label_files = os.listdir(label_dir)  # 展示目标文件夹下所有的文件名
            label_files = list(filter(lambda x: x.endswith('.txt'), label_files))  # 取到所有以.txt结尾的yolo格式文件
            for label_file in label_files:
                with open(os.path.join(label_dir, label_file), 'r') as f:
                    lines = f.readlines()
                for line in lines:
                    line = line.strip().split()
                    class_name = line[0]
                    if class_name in labels_count:
                        labels_count[class_name] += 1
                    else:
                        labels_count[class_name] = 1
                    if class_name in labels_dir_count:
                        labels_dir_count[class_name] += 1
                    else:
                        labels_dir_count[class_name] = 1
            print(f'{dir} done!', 'labels_count:', labels_dir_count)

    return labels_count

if __name__ == '__main__':
    root = r'D:\code\yolov8\guaTiao\zhenbaozhu\labels' # dataset为你的YOLOv8格式的数据集
    labels_cnt = labels_count(root)
    print('all sets done! labels_count:', labels_cnt)


In [ ]:
import labelme2yolo